In [ ]:
import datetime as dt

import xarray as xr
import altair as alt
import pandas as pd

from psp.pv import get_irradiance

alt.data_transformers.disable_max_rows()

In [ ]:
# Add this number of hours to the theoretical data.
# This should be 0 but it's useful to play with it.
DELTA = 0

# We assume that in the PV data the timestamp is the beginning of the window.
# We'll use this variable to offset the theoretical value to get the middle of the window.
FREQUENCY_MINUTES = 60

DATA_PATH = "../normal.nc"
# DATA_PATH = "/mnt/storage_b/data/ocf/solar_pv_nowcasting/clients/island/data_hourly_MW_dstfix_clean_v2.nc"
# d = xr.open_dataset('/mnt/storage_b/data/ocf/solar_pv_nowcasting/clients/island/enemalta_pv_hourly_v5.nc')

In [ ]:
data = xr.open_dataset(DATA_PATH)

# TODO Generalize to more than 1 PV
data = data.isel(pv_id=0)

lat = data.coords["latitude"].values
lon = data.coords["longitude"].values
print("lat", lat)
print("lon", lon)

data = (
    data.to_dataframe()
    .reset_index()
#     .rename(
#         columns={
            
#              "Hourly PV Generated Units (MW)": "power",
#              "datetimeUTC": "ts",
#              "Total Max Capacity (MW)": "capacity",
#         }
#     )
    .rename(
    columns={
        "timestamp": "ts"
    })
)

In [ ]:
d = data.copy()

display(d)

# print(list(d['ts'].dt.minute.unique()))


dd = d.copy()

d = d[["ts", "power", "capacity"]]
display(d.describe())

# Normalize by capacity
d["power"] = d["power"] / d["capacity"]

del d["capacity"]

print("Data")
display(d)

# We do the same thing but with pvlib

d2 = (
    get_irradiance(
        lat=lat,
        lon=lon,
        tilt=35,
        orientation=180,
        timestamps=d["ts"] + dt.timedelta(minutes=FREQUENCY_MINUTES / 2),
    )[["poa_global"]]
    .reset_index(drop=True)
    .rename(columns={"poa_global": "power"})
)

d2["ts"] = d["ts"]

print("Theoretical")
display(d2)

# poa_global = irr['poa_global'].reset_index(drop=True)

# d['poa_global'] = poa_global

# def


def compute_mean_hour(d: pd.DataFrame, win_days=1) -> pd.DataFrame:

    # Compute the power times the hour of day.
    d["power-hour"] = (
        d["power"] * (d["ts"] - pd.to_datetime(d["ts"].dt.date)).dt.total_seconds() / 60.0 / 60.0
    )

    # Sum it per day. Also sum the "power" by itself.
    d = d[["power-hour", "power", "ts"]].groupby([pd.Grouper(key="ts", freq="1D")]).sum()

    d = d.reset_index()

    # Mean hour, weighted by power. This value should change *smoothly* over the year.
    d["mean-hour"] = d["power-hour"] / d["power"]

    # Make sure we are sorted (necessary for the the rolling window)
    d = d.sort_values(["ts"])

    # Smooth the whole thing in time.
    d["mean-hour"] = (
        d[["ts", "mean-hour"]]
        .set_index("ts")
        .rolling(f"{win_days}D", center=True, min_periods=1, closed="both")
        .mean()
        .reset_index(drop=True)
    )

    return d


d = compute_mean_hour(d, win_days=10)
d2 = compute_mean_hour(d2)
#
d2["mean-hour"] += DELTA

d["which"] = "data"
d2["which"] = "theoretical"

d = pd.concat([d, d2])

print("Everything together")
display(d)


print("The two curves should line up perfectly, up to a constant, which you can tweak.")
chart = (
    alt.Chart(d)
    .mark_line()
    .encode(
        x="ts",
        y=alt.Y("mean-hour", scale=alt.Scale(zero=False)),
        #         color="year(ts):N",
        color="which",
    )
    .properties(width=1000, height=200)
)

display(chart)

In [ ]:
d = data.copy()
d["power"] = d["power"] / d["capacity"]
display(d)

chart = (
    alt.Chart(data)
    .mark_line()
    .encode(
        x="hoursminutes(ts)",
        y="power",
        color="year(ts):N",
        facet=alt.Facet("monthdate(ts)", columns=15, spacing=1),
    )
    .properties(width=100, height=100)
)
chart